In [ ]:
import os
import pandas as pd

In [ ]:
os.chdir('C:\\DONNEES\\xplor\\travel')

In [ ]:
booking = pd.read_csv('bookings.csv.bz2', sep='^', header=0, usecols=[9,12,34], compression='bz2')

In [ ]:
searches = pd.read_csv('searches.csv.bz2', sep='^', header=0, usecols=[0,5,6,8], compression='bz2')

In [ ]:
searches['Date'] = pd.to_datetime(searches['Date'], errors='coerce')

In [ ]:
searches['monthSearch'] = searches['Date'].dt.month

In [ ]:
##### Number of lines #####

In [ ]:
lineCountBooking = len(booking)

In [ ]:
lineCountSearches = len(searches)

In [ ]:
print("There are : "+str(lineCountBooking)+" lines in file bookings \n and "+str(lineCountSearches)+" lines in file searches")

In [ ]:
booking['arr_port'] = booking['arr_port'].str.strip()

In [ ]:
##### top 10 arrival airports in the world in 2013 #####

In [ ]:
nbPaxbyAirport = pd.DataFrame(booking.pivot_table(index=['arr_port'], values='pax', aggfunc=sum, fill_value=0))

In [ ]:
top10Airports = pd.DataFrame((nbPaxbyAirport.sort_values(['pax'], axis=0, ascending=0)).head(10))

In [ ]:
top10Airports.to_csv('top10Airports.txt', sep='\t')

In [ ]:
top10Airports = pd.read_csv('top10Airports.txt', sep='\t', header=0)

In [ ]:
airports_geonames = pd.read_csv('airports_geonames_only_clean.csv', sep='^', header=None, names=['iata_code','airport_name','city_code','coutry_code','country','lat','long'])

In [ ]:
top10Airports = pd.merge(top10Airports, airports_geonames, how='left', left_on='arr_port', right_on='iata_code')

In [ ]:
airportRankNames = top10Airports.loc[: ,:"airport_name"]

In [ ]:
airportRankNames.drop('iata_code', axis=1, inplace=True)

In [ ]:
airports_geonames['country'] = airports_geonames['country'].str.strip().str.upper()
airports_geonames['airport_name'] = airports_geonames['airport_name'].str.strip().str.upper()

In [ ]:
##### Searches to Spain #####

In [ ]:
spain = airports_geonames[airports_geonames['country']=='SPAIN']

In [ ]:
#Malaga AGP, Madrid MAD or Barcelona BCN
search3Spain = searches.query('Destination == ["AGP","MAD","BCN"]')

In [ ]:
#search by month
nbSearchSpain = pd.crosstab(search3Spain.monthSearch, search3Spain.Destination)

In [ ]:
## personnaly I prefer bargraph for the number of occurences
#esPlot = nbSearchSpain.plot.bar()
esPlot = nbSearchSpain.plot(title='Number of searches by month')
esPlot.set_ylabel('Nb of searches')
esPlot.set_xlabel('Month')
esFig = esPlot.get_figure()
esFig.savefig('searchSpainCurve')

In [ ]:
##### Matching search and booking #####
##### to improve when having large file ... #####

In [ ]:
searches['s2bKey'] = searches['Origin'].str.strip()+'--'+searches['Destination'].str.strip()

In [ ]:
booking['s2bKey'] = booking['dep_port'].str.strip()+'--'+booking['arr_port'].str.strip()

In [ ]:
Search2Booking = pd.merge(searches, booking, how='left', on='s2bKey')

In [ ]:
Search2Booking['searchNbook'] =  pd.notnull(Search2Booking['dep_port'])*1

In [ ]:
Search2csv = Search2Booking.iloc[: , [0,1,2,3,9]]

In [ ]:
Search2csv.to_csv('searchesWithBookingFlag.csv', sep='\t', index=0)

In [ ]:
##### JSON output without webservice #####

In [ ]:
def topNAirports(n):
    if n > 0:
        try :
            rankAirports = pd.DataFrame((nbPaxbyAirport.sort_values(['pax'], axis=0, ascending=0)).head(n))
        except :
            raf = 'raf'
            rankAirports = print('Expect integer value')
    else:
        rankAirports = print('Expect integer value > to 0')
        
    rankAirports.to_csv('rankAirports.txt', sep='\t')
    rankAirports = pd.read_csv('rankAirports.txt', sep='\t', header=0)
    os.remove('rankAirports.txt')
    return rankAirports.to_json(orient='index')

In [ ]:
topNAirports(10)